## 表形式データの再形成
- <m-b>ピボット</m-b>: 列の値に基づいてデータを再形成する処理
- <m-b>アンピボット</m-b>: 逆の処理

In [2]:
import pandas as pd

tips = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/tips.csv",
    dtype={
        "sex": "category",
        "smoker": "category",
        "day": "category",
        "time": "category",
    },
)
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [28]:
tips.dtypes

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object

In [59]:
tmp = tips.pivot_table(
    index="smoker",
    columns="time",
    values="total_bill",
    observed=True,
)
tmp

time,Dinner,Lunch
smoker,,
No,20.095660,17.050889
Yes,21.859429,17.399130


In [60]:
print(tmp.index)
print(tmp.columns)

CategoricalIndex(['No', 'Yes'], categories=['No', 'Yes'], ordered=False, dtype='category', name='smoker')
CategoricalIndex(['Dinner', 'Lunch'], categories=['Dinner', 'Lunch'], ordered=False, dtype='category', name='time')


In [ ]:
tips.pivot_table(
    index="smoker",
    columns="time",
    values="total_bill",
    observed=True,
    aggfunc="median",
)

time,Dinner,Lunch
smoker,,
No,18.265,15.95
Yes,20.695,16.00


In [37]:
tips.pivot_table(
    index="smoker",
    columns="time",
    values="total_bill",
    observed=True,
    aggfunc="max",
    margins=True,
)

time,Dinner,Lunch,All
smoker,,,
No,48.33,41.19,48.33
Yes,50.81,43.11,50.81
All,50.81,43.11,50.81


In [45]:
tips.pivot_table(
    index=["day", "time"],
    columns=["smoker", "sex"],
    values=["size", "total_bill"],
    observed=False,
    aggfunc={"size": "count", "total_bill": "sum"},
)

size                  total_bill                        
smoker          No         Yes              No             Yes        
sex         Female Male Female Male     Female    Male  Female    Male
day  time                                                             
Fri  Dinner      1    2      4    5      22.75   34.95   48.80  129.46
     Lunch       1    0      3    3      15.98    0.00   39.78   34.16
Sat  Dinner     13   32     15   27     247.05  637.73  304.00  589.62
     Lunch       0    0      0    0       0.00    0.00    0.00    0.00
Sun  Dinner     14   43      4   15     291.54  877.34   66.16  392.12
     Lunch       0    0      0    0       0.00    0.00    0.00    0.00
Thur Dinner      1    0      0    0      18.78    0.00    0.00    0.00
     Lunch      24   20      7   10     381.58  369.73  134.53  191.71

In [50]:
# crosstab()関数によるピボット
tips.pivot_table(
    index="day",
    columns="time",
    values="tip",
    aggfunc="count",
    observed=False,
)

time,Dinner,Lunch
day,,
Fri,12,7
Sat,87,0
Sun,76,0
Thur,1,61


In [48]:
pd.crosstab(
    index=tips.loc[:, "day"],
    columns=tips.loc[:, "time"],
    values=tips.loc[:, "tip"],
    aggfunc="count",
)

time,Dinner,Lunch
day,,
Fri,12,7
Sat,87,0
Sun,76,0
Thur,1,61


In [52]:
pd.crosstab(
    index=tips.loc[:, "day"],
    columns=tips.loc[:, "time"],
)

time,Dinner,Lunch
day,,
Fri,12,7
Sat,87,0
Sun,76,0
Thur,1,61


In [54]:
pd.crosstab(
    index=tips.loc[:, "day"],
    columns=tips.loc[:, "time"],
    normalize=True,
)

time,Dinner,Lunch
day,,
Fri,0.049180,0.028689
Sat,0.356557,0.000000
Sun,0.311475,0.000000
Thur,0.004098,0.250000


In [63]:
# アンピボット
pivoted_df = pd.DataFrame(
    [
        ["No", 20, 17],
        ["Yes", 21, 18],
    ],
    columns=["smoker", "Dinner", "Lunch"]
)
pivoted_df.melt(
    id_vars=["smoker"],
    value_vars=["Dinner", "Lunch"],
    value_name="total_bill",
)

,smoker,variable,total_bill
0,No,Dinner,20
1,Yes,Dinner,21
2,No,Lunch,17
3,Yes,Lunch,18


## スタックとアンスタック
- <m-b>スタック</m-b>: 2次元以上のデータを1次元に積み上げる処理
- <m-b>アンスタック</m-b>: 階層化させたインデックスを列に展開する処理

In [64]:
tips_stacked = tips.stack()
tips_stacked.head(14)

0  total_bill     16.99
   tip             1.01
   sex           Female
   smoker            No
   day              Sun
   time          Dinner
   size               2
1  total_bill     10.34
   tip             1.66
   sex             Male
   smoker            No
   day              Sun
   time          Dinner
   size               3
dtype: object

In [65]:
tips_stacked.dtypes

dtype('O')

In [66]:
tips_stacked.index[:14]

MultiIndex([(0, 'total_bill'),
            (0,        'tip'),
            (0,        'sex'),
            (0,     'smoker'),
            (0,        'day'),
            (0,       'time'),
            (0,       'size'),
            (1, 'total_bill'),
            (1,        'tip'),
            (1,        'sex'),
            (1,     'smoker'),
            (1,        'day'),
            (1,       'time'),
            (1,       'size')],
           )

In [67]:
tips_stacked.unstack()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.5,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.0,Female,Yes,Sat,Dinner,2
241,22.67,2.0,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [3]:
# MultiIndex
groupby_time = tips.groupby("time", observed=False)[
    ["total_bill", "tip"]
].agg(("mean", "median"))
groupby_time

total_bill               tip       
             mean  median      mean median
time                                      
Dinner  20.797159  18.390  3.102670   3.00
Lunch   17.168676  15.965  2.728088   2.25

In [4]:
groupby_time.columns

MultiIndex([('total_bill',   'mean'),
            ('total_bill', 'median'),
            (       'tip',   'mean'),
            (       'tip', 'median')],
           )

In [70]:
groupby_time.columns.names = ("value", "agg",)
groupby_time

value  total_bill               tip       
agg          mean  median      mean median
time                                      
Dinner  20.797159  18.390  3.102670   3.00
Lunch   17.168676  15.965  2.728088   2.25

In [71]:
groupby_time.columns

MultiIndex([('total_bill',   'mean'),
            ('total_bill', 'median'),
            (       'tip',   'mean'),
            (       'tip', 'median')],
           name=('value', 'agg'))

In [73]:
groupby_time.columns.names

FrozenList(['value', 'agg'])

In [82]:
groupby_time_stacked = groupby_time.stack(0, future_stack=True)
groupby_time_stacked

agg                     mean  median
time   value                        
Dinner total_bill  20.797159  18.390
       tip          3.102670   3.000
Lunch  total_bill  17.168676  15.965
       tip          2.728088   2.250

In [90]:
groupby_time.stack("agg", future_stack=True)

value          total_bill       tip
time   agg                         
Dinner mean     20.797159  3.102670
       median   18.390000  3.000000
Lunch  mean     17.168676  2.728088
       median   15.965000  2.250000

In [83]:
groupby_time_stacked.index

MultiIndex([('Dinner', 'total_bill'),
            ('Dinner',        'tip'),
            ( 'Lunch', 'total_bill'),
            ( 'Lunch',        'tip')],
           names=['time', 'value'])

In [84]:
groupby_time_stacked.index.names

FrozenList(['time', 'value'])

In [87]:
groupby_time_stacked.unstack(0)

agg              mean            median        
time           Dinner      Lunch Dinner   Lunch
value                                          
total_bill  20.797159  17.168676  18.39  15.965
tip          3.102670   2.728088   3.00   2.250

In [88]:
groupby_time_stacked.unstack("value")

agg          mean               median      
value  total_bill       tip total_bill   tip
time                                        
Dinner  20.797159  3.102670     18.390  3.00
Lunch   17.168676  2.728088     15.965  2.25

In [91]:
# ダミー変数
pd.get_dummies(tips.loc[:, "day"])

,Fri,Sat,Sun,Thur
0,False,False,True,False
1,False,False,True,False
2,False,False,True,False
3,False,False,True,False
4,False,False,True,False
...,...,...,...,...
239,False,True,False,False
240,False,True,False,False
241,False,True,False,False
242,False,True,False,False


In [93]:
pd.get_dummies(tips)

,total_bill,tip,size,sex_Female,sex_Male,smoker_No,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch
0,16.99,1.01,2,True,False,True,False,False,False,True,False,True,False
1,10.34,1.66,3,False,True,True,False,False,False,True,False,True,False
2,21.01,3.50,3,False,True,True,False,False,False,True,False,True,False
3,23.68,3.31,2,False,True,True,False,False,False,True,False,True,False
4,24.59,3.61,4,True,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,3,False,True,True,False,False,True,False,False,True,False
240,27.18,2.00,2,True,False,False,True,False,True,False,False,True,False
241,22.67,2.00,2,False,True,False,True,False,True,False,False,True,False
242,17.82,1.75,2,False,True,True,False,False,True,False,False,True,False


In [94]:
pd.get_dummies(tips, columns=["smoker", "time"])

,total_bill,tip,sex,day,size,smoker_No,smoker_Yes,time_Dinner,time_Lunch
0,16.99,1.01,Female,Sun,2,True,False,True,False
1,10.34,1.66,Male,Sun,3,True,False,True,False
2,21.01,3.50,Male,Sun,3,True,False,True,False
3,23.68,3.31,Male,Sun,2,True,False,True,False
4,24.59,3.61,Female,Sun,4,True,False,True,False
...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,Sat,3,True,False,True,False
240,27.18,2.00,Female,Sat,2,False,True,True,False
241,22.67,2.00,Male,Sat,2,False,True,True,False
242,17.82,1.75,Male,Sat,2,True,False,True,False


In [95]:
# 要素の展開
students = pd.DataFrame(
    {
        "名前": ["木村", "山田"],
        "科目": [
            ["国語", "英語"],
            ["国語", "英語", "数学"],
        ],
        "得点": [
            [92, 87],
            [85, 96, 88],
        ]
    }
)
students

,名前,科目,得点
0,木村,"[国語, 英語]","[92, 87]"
1,山田,"[国語, 英語, 数学]","[85, 96, 88]"


In [103]:
students.dtypes

名前    object
科目    object
得点    object
dtype: object

In [96]:
students.loc[:, "得点"]

0        [92, 87]
1    [85, 96, 88]
Name: 得点, dtype: object

In [97]:
students.loc[:, "得点"].explode()

0    92
0    87
1    85
1    96
1    88
Name: 得点, dtype: object

In [99]:
students.explode("科目")

,名前,科目,得点
0,木村,国語,"[92, 87]"
0,木村,英語,"[92, 87]"
1,山田,国語,"[85, 96, 88]"
1,山田,英語,"[85, 96, 88]"
1,山田,数学,"[85, 96, 88]"


In [101]:
students.explode(["科目", "得点"])

,名前,科目,得点
0,木村,国語,92
0,木村,英語,87
1,山田,国語,85
1,山田,英語,96
1,山田,数学,88


In [102]:
students.explode(["名前", "科目", "得点"])

ValueError: columns must have matching element counts